In [1]:
from pythonosc import udp_client
from pythonosc import dispatcher
from pythonosc import osc_server
import threading
import time
from random import choice
import signal


# make sure servers are shut down properly when forced close python
def signal_handler(sig, frame):
    print("Shutting down server...")
    server.shutdown()  # Shut down the server gracefully
    print("Shutdown completed")
signal.signal(signal.SIGINT, signal_handler)
signal.signal(signal.SIGTERM, signal_handler)

def receive_handler(address, *args):
    print(f"Received response from {address}: {args}")
    global last_response_time
    last_response_time = time.time()

def send_test_messages(client, test_address, interval=10):
    while True:
        client.send_message(test_address, "Test Message")
        time.sleep(interval)  # Send a test message every 'interval' seconds

measurement_address = "/bruQner/measurement_result"
test_request_address = "/bruQner/connection_test/request"
test_response_address = "/bruQner/connection_test/response"

measurement_results = [1, 2, 3, 4]

if __name__ == "__main__":

    # Setup ports and IP
    target_ip = "172.26.48.1"  # receiving computers IP 
    target_port = 12345  # receiving computers listening port
    listen_port = 12347
    
    # Setup handling of test responses 
    disp = dispatcher.Dispatcher()
    disp.map(test_response_address, receive_handler)  # Expecting responses on /response

    # Setup and run server to receive test responses
    server = osc_server.ThreadingOSCUDPServer(('0.0.0.0', listen_port), disp)
    server_thread = threading.Thread(target=server.serve_forever)
    server_thread.start()

    # Setup clientside (to send out measurements and test requests)
    client = udp_client.SimpleUDPClient(target_ip, target_port)

    # send test messages every 10s forever
    send_test_messages(client, test_request_address, interval=2)
    
    # send measurement result every 1s forever
    while True:
        client.send_message(measurement_address, choice(measurement_results))
        time.sleep(1)  # Adjust delay as needed for your performance

Shutting down server...
